# MIDI Generation with Keras

The code below creates an LSTM RNN, and creates a model using various midi files as training data. The model generated is then used to produce some tunes... hopefully.

In [ ]:
import pickle
from datetime import datetime
from pathlib import Path
from random import randint

import music21 as m21
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

## Define paths

In [ ]:
project_folder = Path('/home/tony/repos/rnn_midi_generator')

checkpoints_folder = project_folder / 'checkpoints'
data_folder = project_folder / 'data'
output_folder = project_folder / 'output'

midi_folder = Path('/c/Users/Josh/Downloads/50000 MIDI FILES/Classical')

## Gather training data
Scan `midi_folder` for `.mid` files and parse each one, storing the result in the `data` folder

In [ ]:
number_of_files_parsed = 0
notes = []

for file in midi_folder.glob('*.mid'):
    midi = m21.converter.parse(file)

    try:  # file has instrument parts
        s2 = m21.instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except:  # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, m21.note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, m21.chord.Chord):
            # notes.append('.'.join(str(n) for n in element.normalOrder))
            notes.extend([str(note.pitch) for note in element._notes])
    number_of_files_parsed += 1

print(f'Number of midi files parsed: {number_of_files_parsed}')

with open(data_folder / 'notes', 'wb') as filepath:
    pickle.dump(notes, filepath)

## Prepare training data

### Create note:int mapping
Create a dictionary mapping each note to an integer representing its position in an ordered list of all the notes.

In [ ]:
pitchnames = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

### Create input sequences and corresponding outputs
The notes are mapped to their integer representation for the neural network to be able to process them.

In [ ]:
network_input = []
network_output = []
sequence_length = 30
notes_length = len(notes)
notes_as_ints = list(map(lambda x: note_to_int[x], notes))

print(f'Number of notes: {notes_length}')

for i in range(notes_length - sequence_length):
    sequence_in = notes_as_ints[i:i + sequence_length]
    sequence_out = notes_as_ints[i + sequence_length]
    network_input.append(sequence_in)
    network_output.append(sequence_out)

### Transform input and output
Reshape the `network_input` into a format compatible with LSTM layers and normalize. One-hot encode the `network_output`.

In [ ]:
pitchnames_length = len(pitchnames)
print(f'Number of unique notes: {pitchnames_length}')
normalized_network_input = np.reshape(
    network_input, 
    (len(network_input), sequence_length, 1),
)
normalized_network_input = normalized_network_input / float(pitchnames_length)
network_output = to_categorical(network_output)

## Create the model

Make a sequential model with several LSTM layers, 2 dense layers, ...

LSTM is used as the data is time-dependent.

In [ ]:
model = Sequential()
model.add(LSTM(512,
               input_shape=(normalized_network_input.shape[1],
                            normalized_network_input.shape[2]),
               return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(pitchnames_length))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(model.summary())

## Train the model

This might take a while...

### Create model checkpoint

In [ ]:
checkpoint_filepath= checkpoints_folder / 'model_{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(
    checkpoint_filepath.absolute,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min',
)

callbacks_list = [checkpoint]

### Fit the model

In [ ]:
model.fit(
    normalized_network_input,
    network_output,
    batch_size=64,
    epochs=2,
    callbacks=callbacks_list,
)

## Predict

### Choose random start pattern from input

In [ ]:
start = randint(0, len(network_input) - 1)
pattern = network_input[start]

### Create int:note mapping

In [ ]:
int_to_note = dict((v, k) for k, v in note_to_int.items())

### Predict 500 notes

In [ ]:
prediction_output = []

for i in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(pitchnames_length)
    
    predicted_note = model.predict(prediction_input)
    index = np.argmax(predicted_note)
    note = int_to_note[index]
    prediction_output.append(note)

    pattern.append(index)
    pattern = pattern[1:]

### Create MIDI file from predicted output

In [ ]:
offset = 0
output_notes = []

for note, velocity in prediction_output:
    new_note = m21.note.Note(note)
    new_note.offset = offset
    new_note.storedInstrument = m21.instrument.Piano()
    output_notes.append(new_note)
    offset += 0.5  # increase offset each iteration so that notes do not stack

timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
midi_stream = m21.stream.Stream(output_notes)
midi_stream.write('midi', fp=output_folder / f'{timestamp}.mid')